<a href="https://colab.research.google.com/github/Haripriya-Mahajan/AI-Model-Comparison-Tool/blob/main/AIModelComparisonTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

!pip install PyPDF2 pandas requests

import os
import pickle
import PyPDF2
import pandas as pd
import re
import requests

# ===============================
# CONFIG
# ===============================
CACHE_DIR = "/content/drive/MyDrive/LLM_Project_Cache"
os.makedirs(CACHE_DIR, exist_ok=True)

PDF_CACHE_PATH = os.path.join(CACHE_DIR, "pdf_texts.pkl")
COMPRESSED_CACHE_PATH = os.path.join(CACHE_DIR, "pdf_texts_compressed.pkl")
CSV_CACHE_PATH = os.path.join(CACHE_DIR, "benchmark_data.pkl")
API_CACHE_PATH = os.path.join(CACHE_DIR, "aa_api_models.pkl")

# ===============================
# PDF EXTRACTION
# ===============================
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, "rb") as f:
            reader = PyPDF2.PdfReader(f, strict=False)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
    except:
        print(f"⚠️ Skipping PDF: {pdf_path}")
    return text

# ===============================
# SCALEDOWN COMPRESSION
# ===============================
def compress_texts_with_scaledown(texts, api_key):
    compressed = []
    headers = {"x-api-key": api_key, "Content-Type": "application/json"}
    for t in texts:
        payload = {
            "context": t[:12000],  # safety cap
            "prompt": "Compress for AI research use",
            "scaledown": {"rate": "auto"}
        }
        try:
            r = requests.post("https://api.scaledown.xyz/compress/raw/",
                              headers=headers, json=payload, timeout=60)
            if r.status_code == 200 and "compressed_prompt" in r.json():
                compressed.append(r.json()["compressed_prompt"])
            else:
                compressed.append(t)
        except:
            compressed.append(t)
    return compressed

# ===============================
# LOAD OR UPLOAD PDFs
# ===============================
if os.path.exists(PDF_CACHE_PATH):
    with open(PDF_CACHE_PATH, "rb") as f:
        pdf_texts = pickle.load(f)
    print("✅ Loaded cached PDFs")
else:
    from google.colab import files
    print("📤 Upload PDF files")
    uploaded = files.upload()
    pdf_texts = []
    for file_name in uploaded:
        if file_name.lower().endswith(".pdf"):
            pdf_texts.append(extract_text_from_pdf(file_name))
    with open(PDF_CACHE_PATH, "wb") as f:
        pickle.dump(pdf_texts, f)
    print("✅ PDFs cached")


SCALEDOWN_API_KEY = input("Enter Scaledown API key (or press Enter to skip compression): ").strip()
if SCALEDOWN_API_KEY:
    if os.path.exists(COMPRESSED_CACHE_PATH):
        with open(COMPRESSED_CACHE_PATH, "rb") as f:
            pdf_texts = pickle.load(f)
        print("✅ Loaded cached compressed PDF texts")
    else:
        print("🌐 Compressing PDF texts with Scaledown API...")
        pdf_texts = compress_texts_with_scaledown(pdf_texts, SCALEDOWN_API_KEY)
        with open(COMPRESSED_CACHE_PATH, "wb") as f:
            pickle.dump(pdf_texts, f)
        print("✅ Compressed texts cached")

# ===============================
# LOAD OR UPLOAD CSV BENCHMARK
# ===============================
if os.path.exists(CSV_CACHE_PATH):
    with open(CSV_CACHE_PATH, "rb") as f:
        benchmark_df = pickle.load(f)
    print("✅ Loaded cached benchmark CSV")
else:
    from google.colab import files
    print("📤 Upload benchmark CSV")
    uploaded = files.upload()
    csv_file = [f for f in uploaded if f.lower().endswith(".csv")][0]
    benchmark_df = pd.read_csv(csv_file)
    with open(CSV_CACHE_PATH, "wb") as f:
        pickle.dump(benchmark_df, f)
    print("✅ CSV cached")

# ===============================
# ARTIFICIAL ANALYSIS API
# ===============================
API_KEY = input("Enter Artificial Analysis API key (or press Enter to skip): ").strip()
api_df = pd.DataFrame()
if API_KEY:
    if os.path.exists(API_CACHE_PATH):
        with open(API_CACHE_PATH, "rb") as f:
            api_df = pickle.load(f)
        print("✅ Loaded cached API data")
    else:
        print("🌐 Fetching data from Artificial Analysis API...")
        url = "https://artificialanalysis.ai/api/v2/data/llms/models"
        headers = {"x-api-key": API_KEY}
        r = requests.get(url, headers=headers)
        if r.status_code == 200:
            data = r.json()["data"]
            rows = []
            for m in data:
                rows.append({
                    "model": m["name"],
                    "creator": m["model_creator"]["name"],
                    "intelligence": m["evaluations"].get("artificial_analysis_intelligence_index"),
                    "coding": m["evaluations"].get("artificial_analysis_coding_index"),
                    "math": m["evaluations"].get("artificial_analysis_math_index"),
                    "speed": m.get("median_output_tokens_per_second"),
                    "latency": m.get("median_time_to_first_token_seconds")
                })
            api_df = pd.DataFrame(rows)
            with open(API_CACHE_PATH, "wb") as f:
                pickle.dump(api_df, f)
            print("✅ API data cached")
        else:
            print("❌ Failed to fetch API data")

# ===============================
# QUERY HANDLING
# ===============================
CANONICAL_MODELS = {
    "gpt": "GPT-4",
    "openai": "GPT-4",
    "claude": "Claude 3 Opus",
    "gemini": "Gemini 1.5 Pro",
    "google": "Gemini 1.5 Pro",
    "llama": "LLaMA 3"
}

def detect_intent(query):
    q = query.lower()
    if any(k in q for k in ["accuracy", "intelligence", "performance", "reasoning"]):
        return "intelligence"
    if any(k in q for k in ["speed", "fast"]):
        return "speed"
    if "latency" in q:
        return "latency"
    return "research"

# ===============================
# Extract models by family and get API variants
# ===============================
def extract_models(query):
    found = []
    query_lower = query.lower()
    for keyword, canonical_name in CANONICAL_MODELS.items():
        if keyword in query_lower:
            # find all API variants that belong to this canonical family
            variants = api_df[api_df['model'].str.contains(canonical_name, case=False)]
            for v in variants['model']:
                if v not in found:
                    found.append(v)
    return found


def bar(score, scale=2):
    return "█" * (score // scale)

def answer_query(query):
    intent = detect_intent(query)
    mentioned_models = extract_models(query)

    # ---------- COMPARISON MODE ----------
    if mentioned_models and not api_df.empty:
        df = api_df[api_df['model'].isin(mentioned_models)]
        if df.empty:
            return "❌ None of the mentioned canonical models were found in the database."
        display_col = {"intelligence":"intelligence","speed":"speed","latency":"latency"}.get(intent, "intelligence")
        answer = f"\n📊 COMPARISON OF SELECTED MODELS BY {intent.upper()}\n\n"
        for i, row in df.iterrows():
            val = row[display_col]
            bar_val = bar(int(val)) if display_col=="intelligence" and not pd.isna(val) else ""
            if pd.isna(val):
                val = "N/A"
            answer += f"{row['model']:<25} | {val:>7} {bar_val}\n"
        return answer

    # ---------- TOP MODELS MODE ----------
    if intent in ["intelligence","speed","latency"] and not api_df.empty:
        col = {"intelligence":"intelligence","speed":"speed","latency":"latency"}[intent]
        df = api_df.dropna(subset=[col])
        ascending = True if intent=="latency" else False
        df = df.sort_values(col, ascending=ascending).head(10)
        if df.empty:
            return f"❌ No {intent} data available."
        title = {
            "intelligence":"TOP MODELS BY INTELLIGENCE",
            "speed":"FASTEST MODELS (Tokens/sec)",
            "latency":"LOWEST LATENCY MODELS (sec)"
        }[intent]
        answer = f"\n📊 {title} (Artificial Analysis)\n\n"
        for i, row in df.iterrows():
            val = row[col]
            bar_val = bar(int(val)) if intent=="intelligence" else ""
            answer += f"{row['model']:<25} | {val:>7} {bar_val}\n"
        return answer

    # ---------- FALLBACK: RESEARCH ----------
    keywords = re.findall(r"\w+", query.lower())
    pdf_hits = []
    for text in pdf_texts:
        score = sum(text.lower().count(k) for k in keywords)
        if score > 0:
            pdf_hits.append((score, text[:500]))
    pdf_hits.sort(reverse=True)

    csv_hits = benchmark_df[benchmark_df.apply(
        lambda row: any(k in str(row).lower() for k in keywords),
        axis=1
    )]

    answer = "\n📌 RESULTS FROM RESEARCH DATA\n\n"
    if not csv_hits.empty:
        answer += "🔹 Benchmarks:\n"
        answer += csv_hits.head(5).to_string(index=False)
        answer += "\n\n"
    if pdf_hits:
        answer += "🔹 Paper Evidence:\n"
        answer += pdf_hits[0][1] + "\n"
    if csv_hits.empty and not pdf_hits:
        answer += "No relevant information found."
    return answer

# ===============================
# RUN
# ===============================
user_query = input("Enter your query: ")
print("\n" + "="*60)
print(answer_query(user_query))
print("="*60)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Loaded cached PDFs
Enter Scaledown API key (or press Enter to skip compression): 
✅ Loaded cached benchmark CSV
Enter Artificial Analysis API key (or press Enter to skip): aa_oLvRzBgzLlSqPWjaNvNSvgwZMoHQAZcJ
✅ Loaded cached API data
Enter your query: Which model has lower latency, gpt or gemini?


📊 COMPARISON OF SELECTED MODELS BY LATENCY

GPT-4o (Aug '24)          |   0.548 
GPT-4o (May '24)          |    0.53 
GPT-4 Turbo               |    1.24 
GPT-4o (Nov '24)          |   0.436 
GPT-4o mini               |   0.555 
GPT-4                     |   0.951 
GPT-4.1 mini              |   0.462 
GPT-4.1                   |   0.473 
GPT-4.1 nano              |   0.372 
GPT-4o mini Realtime (Dec '24) |     0.0 
GPT-4o Realtime (Dec '24) |     0.0 
GPT-4.5 (Preview)         |     0.0 
GPT-4o (ChatGPT)          |   0.458 
GPT-4o (March 2025, chatgpt-4o-latest) |